In [1]:
import pandas as pd

In [261]:
# Daysim formatted 2014 survey with adjusted weights
trip = pd.read_csv(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\Adjusted\trips_2014_adjusted_wt_daysim.csv')
parcel = pd.read_csv(r'L:\LODES\soundcast_2014_lodes\inputs\buffered_parcels.txt', sep=' ')

#export_trip_hh = pd.merge(export_trip_hh, parcel, left_on='opcl', right_on='parcelid')
#export_trip_hh = pd.merge(export_trip_hh, parcel, left_on='dpcl', right_on='parcelid', suffixes=['_o','_d'])

In [262]:
# Sum employment variables by TAZ

# Take a weighted average by emp, pop, and emp+pop for other variables


In [263]:
# Calculate TAZ totals for households, employment
totals_by_taz = parcel.groupby('taz_p').sum()[[
       'hh_p', u'stugrd_p', u'stuhgh_p', u'stuuni_p', u'empedu_p',
       u'empfoo_p', u'empgov_p', u'empind_p', u'empmed_p', u'empofc_p',
       u'empret_p', u'empsvc_p', u'empoth_p', u'emptot_p']]
totals_by_taz.columns = [i.split('_p')[0]+'_taz_tot' for i in totals_by_taz.columns]
totals_by_taz = totals_by_taz.reset_index()


In [264]:
parcel.columns

Index([u'parcelid', u'xcoord_p', u'ycoord_p', u'sqft_p', u'taz_p', u'lutype_p',
       u'hh_p', u'stugrd_p', u'stuhgh_p', u'stuuni_p', u'empedu_p',
       u'empfoo_p', u'empgov_p', u'empind_p', u'empmed_p', u'empofc_p',
       u'empret_p', u'empsvc_p', u'empoth_p', u'emptot_p', u'parkdy_p',
       u'parkhr_p', u'ppricdyp', u'pprichrp', u'hh_1', u'stugrd_1',
       u'stuhgh_1', u'stuuni_1', u'empedu_1', u'empfoo_1', u'empgov_1',
       u'empind_1', u'empmed_1', u'empofc_1', u'empret_1', u'empsvc_1',
       u'empoth_1', u'emptot_1', u'parkdy_1', u'parkhr_1', u'ppricdy1',
       u'pprichr1', u'nodes1_1', u'nodes3_1', u'nodes4_1', u'tstops_1',
       u'nparks_1', u'aparks_1', u'hh_2', u'stugrd_2', u'stuhgh_2',
       u'stuuni_2', u'empedu_2', u'empfoo_2', u'empgov_2', u'empind_2',
       u'empmed_2', u'empofc_2', u'empret_2', u'empsvc_2', u'empoth_2',
       u'emptot_2', u'parkdy_2', u'parkhr_2', u'ppricdy2', u'pprichr2',
       u'nodes1_2', u'nodes3_2', u'nodes4_2', u'tstops_2', u'nparks_

In [265]:
# Calculate a weighted average for each TAZ
# based on the parcel buffers
# This avoids bias based on TAZ boundaries

# Weighted by parcel population, parcel employment, and combination of both
# For 1st buffer (0.25 miles?)

wt_fields = ['stugrd_',
       u'stuhgh_', u'stuuni_', u'empedu_', u'empfoo_', u'empgov_',
       u'empind_', u'empmed_', u'empofc_', u'empret_', u'empsvc_',
       u'empoth_', u'emptot_', u'parkdy_', u'parkhr_', u'ppricdy',
       u'pprichr','tstops_']

transit_fields = ['dist_lbus', u'dist_ebus', u'dist_crt', u'dist_fry', u'dist_lrt', u'dist_park']

for field in wt_fields:
    
    # Get numerator of weights (values within buffered areas)
    # to be divided by denominator of total hh/emp/hh+emp
    
    # Weight by parcel population, for 0.25 and 0.5 mile buffers
    parcel[field+'hh_wt1_tot'] = parcel[field+'1']*parcel['hh_p']    # .25 mile
    parcel[field+'hh_wt2_tot'] = parcel[field+'2']*parcel['hh_p']    # .5 mile
    
    # Weight by total parcel employment
    parcel[field+'emp_wt1_tot'] = parcel[field+'1']*parcel['emptot_p']    # .25 mile
    parcel[field+'emp_wt2_tot'] = parcel[field+'2']*parcel['emptot_p']    # .5 mile
    
    # Weight by households + employment
    parcel[field+'hhemp_wt1_tot'] = parcel[field+'1']*(parcel['emptot_p']+parcel['hh_p'])# .25 mile
    parcel[field+'hhemp_wt2_tot'] = parcel[field+'2']*(parcel['emptot_p']+parcel['hh_p'])# .5 mile

for field in transit_fields:
    
    parcel[field+'_hh_wt_tot'] = parcel[field]*parcel['hh_p']
    parcel[field+'_emp_wt_tot'] = parcel[field]*parcel['emptot_p']
    parcel[field+'_hhemp_wt_tot'] = parcel[field]*(parcel['emptot_p']+parcel['hh_p'])

In [266]:
# Compute weighted average for TAZ based on 

parcel['emptot_plus_hh'] = parcel['emptot_p']+parcel['hh_p']

for field in wt_fields:
    print field
    # Weights based on household size
    totals_by_taz[field+'hh_wt1_avg'] = parcel.groupby('taz_p').sum()[field+'hh_wt1_tot']/parcel.groupby('taz_p').sum()['hh_p']
    totals_by_taz[field+'hh_wt2_avg'] = parcel.groupby('taz_p').sum()[field+'hh_wt2_tot']/parcel.groupby('taz_p').sum()['hh_p']
    
    # Weights based on employment
    totals_by_taz[field+'emp_wt1_avg'] = parcel.groupby('taz_p').sum()[field+'emp_wt1_tot']/parcel.groupby('taz_p').sum()['emptot_p']
    totals_by_taz[field+'emp_wt2_avg'] = parcel.groupby('taz_p').sum()[field+'emp_wt2_tot']/parcel.groupby('taz_p').sum()['emptot_p']
    
    # weights based on housheholds + employment
    totals_by_taz[field+'hhemp_wt1_avg'] = parcel.groupby('taz_p').sum()[field+'hhemp_wt1_tot']/parcel.groupby('taz_p').sum()['emptot_plus_hh']
    totals_by_taz[field+'hhemp_wt2_avg'] = parcel.groupby('taz_p').sum()[field+'hhemp_wt2_tot']/parcel.groupby('taz_p').sum()['emptot_plus_hh']
    
for field in transit_fields:
    totals_by_taz[field+'_hh_wt_avg'] = parcel.groupby('taz_p').sum()[field+'_hh_wt_tot']/parcel.groupby('taz_p').sum()['hh_p']
    totals_by_taz[field+'_emp_wt_avg'] = parcel.groupby('taz_p').sum()[field+'_emp_wt_tot']/parcel.groupby('taz_p').sum()['emptot_p']
    totals_by_taz[field+'_hhemp_wt_avg'] = parcel.groupby('taz_p').sum()[field+'_hhemp_wt_tot']/parcel.groupby('taz_p').sum()['emptot_plus_hh']

stugrd_
stuhgh_
stuuni_
empedu_
empfoo_
empgov_
empind_
empmed_
empofc_
empret_
empsvc_
empoth_
emptot_
parkdy_
parkhr_
ppricdy
pprichr
tstops_


In [273]:
# Fill NaNs with zeros - for zones with 0 hh or emp, divide by 0 results in NaN
totals_by_taz = totals_by_taz.fillna(0)

# Join totals by taz to the trip file
trip = pd.read_csv(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\Adjusted\trips_2014_adjusted_wt_daysim.csv')

In [274]:
# Exclude trips to or from external zones
trip = trip[trip['otaz'] != 0]
trip = trip[trip['dtaz'] != 0]

In [290]:
df = pd.merge(trip, totals_by_taz, left_on='otaz', right_on='taz_p', how='left')
df = pd.merge(df, totals_by_taz, left_on='dtaz', right_on='taz_p', how='left', suffixes=['_o','_d'])
df = df.fillna(0)

In [291]:
# Join transit access mode to data to split transit mode into walk and auto-access
tour = pd.read_csv(r'R:\SoundCastDocuments\2014Estimation\Files_From_Mark_2014\new_weights_10_28_16\skims_attached\tourP14.dat',
                   delim_whitespace=True)

df = pd.merge(df, tour[['hhno','pno','day','tour','tmodetp']], on=['hhno','pno','day','tour'], how='left')

In [292]:
df.to_csv(r'R:\4K\2014\Trip Generation\Trip Rates\2014-new-adjusted\trip_2014_adjusted_lu_vars.csv', index=False)